In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.layers import Attention
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import shapiro
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from numpy.lib.stride_tricks import as_strided
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

from keras.models import Sequential, Model
from keras.layers import Conv1D, AveragePooling1D, Flatten, Activation, Conv2D, BatchNormalization, Input, Concatenate, Dense, MaxPooling1D, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


In [3]:
file_path = 'data_extracted/1.csv'
data = pd.read_csv(file_path, header=None)

features = [1]

for x in range(1, 11):
    lag_name = "lag_" + str(x)
    data[lag_name] = data[1].shift(x)
    features.append(lag_name)

data.dropna(inplace=True)

time = data[0]
y = data[2].astype(int)
X = data[features]


In [4]:
X.head

<bound method NDFrame.head of                1     lag_1     lag_2     lag_3     lag_4     lag_5     lag_6  \
10     -0.192859 -0.111333 -0.043488  0.004215  0.027046  0.022390 -0.010066   
11     -0.280377 -0.192859 -0.111333 -0.043488  0.004215  0.027046  0.022390   
12     -0.365584 -0.280377 -0.192859 -0.111333 -0.043488  0.004215  0.027046   
13     -0.440217 -0.365584 -0.280377 -0.192859 -0.111333 -0.043488  0.004215   
14     -0.496707 -0.440217 -0.365584 -0.280377 -0.192859 -0.111333 -0.043488   
...          ...       ...       ...       ...       ...       ...       ...   
129994  0.126695  0.141051  0.145359  0.142650  0.136523  0.130716  0.128570   
129995  0.100302  0.126695  0.141051  0.145359  0.142650  0.136523  0.130716   
129996  0.061114  0.100302  0.126695  0.141051  0.145359  0.142650  0.136523   
129997  0.009739  0.061114  0.100302  0.126695  0.141051  0.145359  0.142650   
129998 -0.051950  0.009739  0.061114  0.100302  0.126695  0.141051  0.145359   

         

In [3]:
encoder = OneHotEncoder(sparse_output=False)
#y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))


# Normalize the signal values
scaler = MinMaxScaler(feature_range = (0, 1))
#scaler = StandardScaler()

X = scaler.fit_transform(X)
#X_with_time = np.column_stack((time, X))

#X_scaled = X
# Сначала разделяем данные на тренировочные и временные выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False)

# Затем разделяем временные выборки на тестовые и валидационные
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False)

# Затем разделяем временные выборки на тестовые и валидационные
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

Adding lag columns can help in classification

In [135]:
X_train = X_train.flatten()
X_test = X_test.flatten()
X_val = X_val.flatten()

y_train = y_train.flatten()
y_test = y_test.flatten()
y_val = y_val.flatten()

In [127]:
len(X_train)

103999

In [12]:
X_train.shape

(103991, 11)

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, TimeDistributed, Bidirectional

# Define the encoder
encoder_inputs = Input(shape=(None, 1))
encoder = Bidirectional(LSTM(64, return_sequences=True))(encoder_inputs)
encoder = Dropout(0.2)(encoder)
encoder_outputs, state_h, state_c = LSTM(64, return_state=True)(encoder)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None, 3))  # 3 classes: 0, 1, 2
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_outputs = Dropout(0.2)(decoder_outputs)
decoder_dense = TimeDistributed(Dense(3, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Number of classes (0, 1, 2)
num_classes = 3

# Create decoder inputs by shifting the target labels
# Add a start token (e.g., 0) at the beginning of the sequence
start_token = 0  # You can use a special token if needed
decoder_inputs = np.insert(y_train[:-1], 0, start_token)

# One-hot encode the decoder inputs
decoder_inputs_one_hot = np.eye(num_classes)[decoder_inputs]

print("Target Labels:", y_train[:10])
print("Decoder Inputs (shifted):", decoder_inputs[:10])
print("Decoder Inputs (one-hot encoded):")
print(decoder_inputs_one_hot[:10])

Target Labels: 10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: 2, dtype: int32
Decoder Inputs (shifted): [0 0 0 0 0 0 0 0 0 0]
Decoder Inputs (one-hot encoded):
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [20]:
X_train_enc_dec = X_train[1]
print(X_train_enc_dec[:10])
print(X_train_enc_dec.shape)

10   -0.192859
11   -0.280377
12   -0.365584
13   -0.440217
14   -0.496707
15   -0.528726
16   -0.531687
17   -0.503070
18   -0.442609
19   -0.352334
Name: 1, dtype: float64
(103991,)


In [22]:
print(X_train_enc_dec.shape)
print(decoder_inputs_one_hot.shape)
print(y_train.shape)

(103991,)
(103991, 3)
(103991,)


In [27]:
# Add batch dimension
encoder_input_data = np.expand_dims(X_train_enc_dec, axis=0)  # Shape: (1, 50, 1)
decoder_input_data = np.expand_dims(decoder_inputs_one_hot, axis=0)  # Shape: (1, 50, 3)
decoder_target_data = np.expand_dims(y_train, axis=0)  # Shape: (1, 50, 3)

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=1, epochs=50, validation_split=0.2)

ValueError: Training data contains 1 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.

In [28]:
print("Encoder input shape:", encoder_input_data.shape)  # Should be (batch_size, sequence_length, 1)
print("Decoder input shape:", decoder_input_data.shape)  # Should be (batch_size, sequence_length, 3)
print("Decoder target shape:", decoder_target_data.shape)  # Should be (batch_size, sequence_length, 3)

Encoder input shape: (1, 103991)
Decoder input shape: (1, 103991, 3)
Decoder target shape: (1, 103991)
